In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  0


In [1]:
import tensorflow as tf
import numpy as np
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from wrappers import wrapper
from agent_testing import DQNAgent

import matplotlib.pyplot as plt 


env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env = wrapper(env)

states = (84, 84, 4)
actions = env.action_space.n

agent = DQNAgent(states=states, actions=actions, max_memory=100000, double_q=True)
agent.reloadModel()



Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
INFO:tensorflow:Restoring parameters from ./newest_models/model-6217557


In [5]:
%load_ext tensorboard
%tensorboard --logdir ./logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
for i in range(100):
    state = env.reset()
    total_reward = 0
    while True:
        env.render()
        action = agent.run(state=state)
        #print(np.shape(state))
        #plt.imshow(state[:,:,0],cmap = 'gray')
        #plt.show()
        next_state, reward, done, info = env.step(action=action)
        total_reward += reward
        state = next_state
        if done:
            break

/home/gmo/.local/lib/python3.5/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


In [2]:
agent.replay(env,'./newest_models/',20, plot=False)

INFO:tensorflow:Restoring parameters from ./newest_models/model-6217557


/home/gmo/.local/lib/python3.5/site-packages/gym_super_mario_bros/smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 